In [16]:
import os
import sys
import re

import calendar
from datetime import datetime, timedelta
from collections import defaultdict
import time

import pandas as pd 
import numpy as np
import json


In [2]:
df_data = pd.read_csv("C:\\Users\\Melt\\Documents\\SYnthProject\\synthesisOne\\data\\interim\\mergedAll.csv")

C:\Users\Melt\AppData\Local\Temp\ipykernel_11100\1373676572.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv("C:\\Users\\Melt\\Documents\\SYnthProject\\synthesisOne\\data\\interim\\mergedAll.csv")


In [3]:
#Make a copy to work with
df_copy = df_data.copy()

In [4]:
df_copy.head()

,PROJECT_ID,TASK_ID,PM,TASK_TYPE,START_TASK,END_TASK,ASSIGNED,READY_TASK,WORKING_TASK,DELIVERED_TASK,...,DISCREPANCY_HOURLY_RATE,CLIENT_HOURLY_PRICE,MANUFACTURER,MANUFACTURER_SECTOR,MANUFACTURER_INDUSTRY_GROUP,MANUFACTURER_INDUSTRY,MANUFACTURER_SUBINDUSTRY,CLIENT_MIN_QUALITY,CLIENT_WILDCARD,MEETS_CLIENT_QUALITY
0,220138,10988490,RMT,ProofReading,2023-04-06 13:00:00,2021-04-07 16:00:00,2021-04-06 18:56:55,2021-04-07 00:01:16,2021-04-07 04:52:34,2021-04-07 05:40:37,...,0,25,Appcelerate,Information Technology,Software & Services,Application Software,Application Software,7.0,Deadline,True
1,220830,11230024,KMT,Engineering,2023-02-13 09:07:00,2023-02-20 19:00:00,2022-12-22 16:16:06,2023-02-10 13:53:38,2023-02-10 17:19:55,2023-02-15 13:49:19,...,-1,25,InnovateWorks,Consumer Discretionary,Consumer Services,Diversified Consumer Services,Education Services,7.5,Deadline,False
2,220830,11230241,KMT,Engineering,2023-02-09 09:24:00,2023-02-20 17:00:00,2022-12-23 09:43:00,2023-02-08 11:28:07,2023-02-08 11:28:30,2023-02-09 10:17:41,...,-1,25,InnovateWorks,Consumer Discretionary,Consumer Services,Diversified Consumer Services,Education Services,7.5,Deadline,False
3,220830,11229295,KMT,Engineering,2023-02-06 17:19:00,2023-02-20 12:00:00,2022-12-21 10:25:09,2023-02-02 16:07:22,2023-02-02 16:07:25,2023-02-09 09:33:03,...,-1,25,InnovateWorks,Consumer Discretionary,Consumer Services,Diversified Consumer Services,Education Services,7.5,Deadline,False
4,220830,11229318,KMT,Engineering,2023-02-02 13:13:00,2023-02-20 17:00:00,2022-12-21 10:39:33,2023-02-02 16:00:40,2023-02-02 16:00:43,2023-02-03 09:19:37,...,-1,25,InnovateWorks,Consumer Discretionary,Consumer Services,Diversified Consumer Services,Education Services,7.5,Deadline,False


In [5]:
df_copy["START_TASK"] = pd.to_datetime(df_copy['START_TASK'])
df_copy["END_TASK"] = pd.to_datetime(df_copy['END_TASK'])

In [6]:
df_copy.head()

,PROJECT_ID,TASK_ID,PM,TASK_TYPE,START_TASK,END_TASK,ASSIGNED,READY_TASK,WORKING_TASK,DELIVERED_TASK,...,DISCREPANCY_HOURLY_RATE,CLIENT_HOURLY_PRICE,MANUFACTURER,MANUFACTURER_SECTOR,MANUFACTURER_INDUSTRY_GROUP,MANUFACTURER_INDUSTRY,MANUFACTURER_SUBINDUSTRY,CLIENT_MIN_QUALITY,CLIENT_WILDCARD,MEETS_CLIENT_QUALITY
0,220138,10988490,RMT,ProofReading,2023-04-06 13:00:00,2021-04-07 16:00:00,2021-04-06 18:56:55,2021-04-07 00:01:16,2021-04-07 04:52:34,2021-04-07 05:40:37,...,0,25,Appcelerate,Information Technology,Software & Services,Application Software,Application Software,7.0,Deadline,True
1,220830,11230024,KMT,Engineering,2023-02-13 09:07:00,2023-02-20 19:00:00,2022-12-22 16:16:06,2023-02-10 13:53:38,2023-02-10 17:19:55,2023-02-15 13:49:19,...,-1,25,InnovateWorks,Consumer Discretionary,Consumer Services,Diversified Consumer Services,Education Services,7.5,Deadline,False
2,220830,11230241,KMT,Engineering,2023-02-09 09:24:00,2023-02-20 17:00:00,2022-12-23 09:43:00,2023-02-08 11:28:07,2023-02-08 11:28:30,2023-02-09 10:17:41,...,-1,25,InnovateWorks,Consumer Discretionary,Consumer Services,Diversified Consumer Services,Education Services,7.5,Deadline,False
3,220830,11229295,KMT,Engineering,2023-02-06 17:19:00,2023-02-20 12:00:00,2022-12-21 10:25:09,2023-02-02 16:07:22,2023-02-02 16:07:25,2023-02-09 09:33:03,...,-1,25,InnovateWorks,Consumer Discretionary,Consumer Services,Diversified Consumer Services,Education Services,7.5,Deadline,False
4,220830,11229318,KMT,Engineering,2023-02-02 13:13:00,2023-02-20 17:00:00,2022-12-21 10:39:33,2023-02-02 16:00:40,2023-02-02 16:00:43,2023-02-03 09:19:37,...,-1,25,InnovateWorks,Consumer Discretionary,Consumer Services,Diversified Consumer Services,Education Services,7.5,Deadline,False


In [7]:
df_copy["START_TASK"].head()

0   2023-04-06 13:00:00
1   2023-02-13 09:07:00
2   2023-02-09 09:24:00
3   2023-02-06 17:19:00
4   2023-02-02 13:13:00
Name: START_TASK, dtype: datetime64[ns]

In [14]:
# ------------------------
# MOCK HISTORICAL DATA
# ------------------------
data = {
    'task_id': [1, 2, 3, 4],
    'translator': ['Alice', 'Bob', 'Alice', 'Bob'],
    'start': pd.to_datetime(['2024-01-02 10:00', '2024-01-03 11:00', '2024-01-10 10:00', '2024-01-11 12:00']),
    'deadline': pd.to_datetime(['2024-01-04 18:00', '2024-01-05 18:00', '2024-01-12 18:00', '2024-01-13 18:00']),
    'duration_hours': [10, 15, 20, 30],
    'work_start': ['10:00', '10:00', '10:00', '10:00'],
    'work_end': ['20:00', '20:00', '20:00', '20:00'],
    'mon': [1, 1, 1, 1],
    'tue': [1, 1, 1, 1],
    'wed': [1, 1, 1, 1],
    'thu': [1, 1, 1, 1],
    'fri': [1, 1, 1, 1],
    'sat': [0, 0, 0, 0],
    'sun': [0, 0, 0, 0],
}

historical_df = pd.DataFrame(data)

In [9]:
# ------------------------
# STEP 1: Group tasks by week and output JSON
# ------------------------
def get_weekly_task_json(df):
    df['week'] = df['start'].dt.isocalendar().week
    df['year'] = df['start'].dt.isocalendar().year
    grouped = defaultdict(list)
    for _, row in df.iterrows():
        year_week = f"{row['year']}_W{row['week']}"
        grouped[year_week].append(row.drop(['week', 'year']).to_dict())
    return grouped

In [10]:
# ------------------------
# STEP 2: Build translator calendars
# ------------------------
def build_translator_calendars(df):
    calendars = defaultdict(list)
    for _, row in df.iterrows():
        calendars[row['translator']].append((row['task_id'], row['start'], row['start'] + timedelta(hours=row['duration_hours'])))
    return calendars

In [11]:

# ------------------------
# STEP 3: Check translator availability for a task
# ------------------------
def is_translator_available(task_row, translator_calendar):
    task_start = task_row['start']
    task_deadline = task_row['deadline']
    duration = task_row['duration_hours']
    
    work_start = datetime.strptime(task_row['work_start'], "%H:%M").time()
    work_end = datetime.strptime(task_row['work_end'], "%H:%M").time()
    
    weekday_avail = {
        0: task_row['mon'], 1: task_row['tue'], 2: task_row['wed'],
        3: task_row['thu'], 4: task_row['fri'], 5: task_row['sat'], 6: task_row['sun']
    }

    scheduled = translator_calendar[task_row['translator']]

    current_time = task_start
    hours_remaining = duration

    while current_time <= task_deadline and hours_remaining > 0:
        if weekday_avail[current_time.weekday()]:
            start_dt = datetime.combine(current_time.date(), work_start)
            end_dt = datetime.combine(current_time.date(), work_end)

            # Find free blocks (not overlapping with scheduled tasks)
            busy = [(s, e) for (_, s, e) in scheduled if s.date() == current_time.date()]
            busy.sort()

            free_slots = []
            last_end = start_dt
            for b_start, b_end in busy:
                if b_start > last_end:
                    free_slots.append((last_end, b_start))
                last_end = max(last_end, b_end)
            if last_end < end_dt:
                free_slots.append((last_end, end_dt))

            for slot_start, slot_end in free_slots:
                free_hours = (slot_end - slot_start).total_seconds() / 3600
                if free_hours <= 0:
                    continue
                hours_to_assign = min(hours_remaining, free_hours)
                scheduled.append((task_row['task_id'], slot_start, slot_start + timedelta(hours=hours_to_assign)))
                hours_remaining -= hours_to_assign
                if hours_remaining <= 0:
                    break
        current_time += timedelta(days=1)

    return hours_remaining <= 0

In [12]:
# ------------------------
# STEP 4: Assign tasks based on availability from SAT/ML output
# ------------------------
def assign_tasks_with_candidates(task_df, candidates_dict):
    translator_calendars = build_translator_calendars(task_df)
    assignments = []
    
    for _, task_row in task_df.iterrows():
        task_id = task_row['task_id']
        candidate_translators = candidates_dict.get(task_id, [])
        assigned = False

        for translator in candidate_translators:
            task_copy = task_row.copy()
            task_copy['translator'] = translator
            if is_translator_available(task_copy, translator_calendars):
                translator_calendars[translator].append(
                    (task_id, task_copy['start'], task_copy['start'] + timedelta(hours=task_copy['duration_hours']))
                )
                assignments.append(task_copy.to_dict())
                assigned = True
                break

        if not assigned:
            print(f"❌ Task {task_id} could not be assigned to any candidate.")

    return pd.DataFrame(assignments)

In [17]:
# ------------------------
# Example Usage 
# ------------------------
weekly_json = get_weekly_task_json(historical_df)
print(json.dumps(weekly_json, indent=2, default=str))

# Example candidates from SAT/ML model
candidates_example = {
    1: ['Alice', 'Bob'],
    2: ['Bob'],
    3: ['Alice'],
    4: ['Bob', 'Alice'],
}

final_assignments = assign_tasks_with_candidates(historical_df, candidates_example)
print("\nFinal Assigned Tasks:")
print(final_assignments)


{
  "2024_W1": [
    {
      "task_id": 1,
      "translator": "Alice",
      "start": "2024-01-02 10:00:00",
      "deadline": "2024-01-04 18:00:00",
      "duration_hours": 10,
      "work_start": "10:00",
      "work_end": "20:00",
      "mon": 1,
      "tue": 1,
      "wed": 1,
      "thu": 1,
      "fri": 1,
      "sat": 0,
      "sun": 0
    },
    {
      "task_id": 2,
      "translator": "Bob",
      "start": "2024-01-03 11:00:00",
      "deadline": "2024-01-05 18:00:00",
      "duration_hours": 15,
      "work_start": "10:00",
      "work_end": "20:00",
      "mon": 1,
      "tue": 1,
      "wed": 1,
      "thu": 1,
      "fri": 1,
      "sat": 0,
      "sun": 0
    }
  ],
  "2024_W2": [
    {
      "task_id": 3,
      "translator": "Alice",
      "start": "2024-01-10 10:00:00",
      "deadline": "2024-01-12 18:00:00",
      "duration_hours": 20,
      "work_start": "10:00",
      "work_end": "20:00",
      "mon": 1,
      "tue": 1,
      "wed": 1,
      "thu": 1,
      "fri":